## Key Concepts and Glossary References

See: `Machine Learning with Python/notes/glossary-module-2.md`
- Logistic Regression
- Sigmoid Function (Logistic Function)
- Odds, Log Odds (Logit), Odds Ratio
- Decision Boundary, Threshold
- Class Imbalance
- Probability Calibration

We will reference these terms inline in comments where used.

## Run Me First (Setup)
This cell installs/imports dependencies, sets seeds, and configures plotting.

In [ ]:
# Imports: core scientific stack
import numpy as np  # numerical computing (vectors, matrices)
import pandas as pd  # data manipulation with DataFrames

# Imports: plotting libraries
import matplotlib.pyplot as plt  # plotting primitives
import seaborn as sns  # statistical visualizations built on matplotlib

# Imports: scikit-learn utilities
from sklearn.model_selection import train_test_split  # split data into train/test (see glossary: Train/Test Split)
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # encode categories; scale numeric features (see glossary: Feature Scaling)
from sklearn.compose import ColumnTransformer  # apply different transforms to columns
from sklearn.pipeline import Pipeline  # chain preprocessing + model in one object (see glossary: Pipeline)
from sklearn.linear_model import LogisticRegression  # logistic regression classifier (see glossary: Logistic Regression)
from sklearn.metrics import (  # evaluation metrics for classification
    accuracy_score, precision_score, recall_score, f1_score,  # core metrics
    roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, classification_report  # curves + diagnostics
)
from sklearn.calibration import calibration_curve  # probability calibration curve (see glossary: Probability Calibration)
from sklearn.ensemble import RandomForestClassifier  # alternative model for comparison

# Configure deterministic behavior
np.random.seed(42)  # set global NumPy seed for reproducibility

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')  # use a modern seaborn-compatible style
sns.set_palette('deep')  # set a color palette for consistency
%matplotlib inline  # render plots inline in the notebook

# Display options for pandas
pd.set_option('display.max_columns', None)  # show all columns when printing DataFrames
pd.set_option('display.precision', 3)  # set numeric precision for readability

# Print library versions for reproducibility
print('✅ Setup complete!')  # setup confirmation
print(f'NumPy: {np.__version__}')  # report NumPy version
print(f'pandas: {pd.__version__}')  # report pandas version

## Generate Synthetic Telecom Churn Dataset
We create a realistic binary classification dataset with meaningful features.
- Industry context: Telecommunications (contracts, monthly charges, support calls)
- Target: `churn` (1 = left, 0 = stayed)
- Includes categorical and numeric features, correlations, and mild outliers

In [ ]:
# Set sample size for the synthetic dataset (see instruction: 50-200 rows)
n = 150  # number of customers to simulate

# Generate numeric features with realistic ranges
age = np.random.randint(18, 80, size=n)  # customer age in years
tenure_months = np.random.randint(1, 72, size=n)  # months with company (1-71)
monthly_charges = np.random.uniform(20, 120, size=n).round(2)  # monthly fee in dollars
support_calls = np.random.poisson(lam=1.5, size=n)  # count of support tickets last 3 months

# Generate a categorical feature: contract type (monthly, annual, two_year)
contract_type = np.random.choice(['monthly', 'annual', 'two_year'], size=n, p=[0.6, 0.3, 0.1])  # category with probabilities

# Introduce mild outliers for realism (a few high-charge or high-support customers)
outlier_idx = np.random.choice(np.arange(n), size=5, replace=False)  # choose some indices for outliers
monthly_charges[outlier_idx] *= 1.8  # increase charges by 80% for selected customers
support_calls[outlier_idx] += 4  # add extra support calls for the same customers

# Define true underlying coefficients to compute log-odds (see glossary: Log Odds / Logit)
beta_0 = -2.0  # intercept term shifts overall churn prevalence
beta_age = 0.012  # older customers slightly more likely to churn (hypothetical)
beta_tenure = -0.03  # longer tenure reduces churn odds
beta_monthly = 0.02  # higher monthly charges increase churn odds
beta_support = 0.18  # more support calls increase churn odds

# Map categorical effects to coefficients (reference: 'annual' as baseline)
beta_contract = {  # contract type contributions to log-odds
    'annual': 0.0,       # baseline
    'monthly': 0.8,      # monthly contracts churn more
    'two_year': -0.5     # longer contracts churn less
}

# Compute linear score z for each customer (z = θ^T x)
z = (  # linear combination of features before sigmoid (see glossary: Sigmoid Function)
    beta_0
    + beta_age * age
    + beta_tenure * tenure_months
    + beta_monthly * monthly_charges
    + beta_support * support_calls
    + np.array([beta_contract[c] for c in contract_type])
)

# Convert z to probabilities via sigmoid: p = 1 / (1 + exp(-z)) (see glossary: Probability)
p = 1 / (1 + np.exp(-z))  # predicted probability of churn for each customer

# Sample binary target from Bernoulli distribution using probability p (see glossary: Binary Classification)
churn = np.random.binomial(n=1, p=p, size=n)  # 1 = churn, 0 = stay

# Assemble DataFrame
df = pd.DataFrame({  # create a table of all features and target
    'age': age,
    'tenure_months': tenure_months,
    'monthly_charges': monthly_charges,
    'support_calls': support_calls,
    'contract_type': contract_type,
    'churn': churn
})

# Show basic dataset info
print('✅ Synthetic dataset created')  # confirmation message
print(df.head())  # preview first rows
print(df['churn'].value_counts(normalize=True).rename('class_ratio'))  # show class balance (see glossary: Class Imbalance)

## Data Dictionary
- `age`: Customer age in years (numeric)
- `tenure_months`: Number of months with the company (numeric)
- `monthly_charges`: Monthly fee in USD (numeric)
- `support_calls`: Number of support tickets in last 3 months (numeric)
- `contract_type`: Contract category (categorical: monthly, annual, two_year)
- `churn`: Target label (binary: 1 = churned, 0 = stayed)

## Exploratory Data Analysis (EDA)
We explore distributions, relationships, and class imbalance.

In [ ]:
# View basic structure
print(df.shape)  # print number of rows and columns
print(df.dtypes)  # print data types of each column
display(df.describe(include='all'))  # summary stats for numeric + categorical

In [ ]:
# Plot distributions of numeric features
fig, axes = plt.subplots(2, 2, figsize=(10, 8))  # create a 2x2 grid of subplots
sns.histplot(df['age'], bins=20, ax=axes[0, 0])  # histogram of age
axes[0, 0].set_title('Age Distribution')  # add title
sns.histplot(df['tenure_months'], bins=20, ax=axes[0, 1])  # histogram of tenure
axes[0, 1].set_title('Tenure (Months)')  # add title
sns.histplot(df['monthly_charges'], bins=20, ax=axes[1, 0])  # histogram of charges
axes[1, 0].set_title('Monthly Charges ($)')  # add title
sns.histplot(df['support_calls'], bins=15, ax=axes[1, 1])  # histogram of support calls
axes[1, 1].set_title('Support Calls (3 mo)')  # add title
plt.tight_layout()  # adjust layout to prevent overlap
plt.show()  # render the plots

In [ ]:
# Visualize churn by contract_type (categorical vs target)
plt.figure(figsize=(6, 4))  # set figure size
sns.barplot(x='contract_type', y='churn', data=df, estimator=np.mean)  # mean churn rate per category
plt.title('Churn Rate by Contract Type')  # add title
plt.ylabel('Mean Churn Rate')  # label y-axis
plt.show()  # render plot

In [ ]:
# Correlation matrix for numeric features vs churn (see glossary: Correlation)
corr = df[['age', 'tenure_months', 'monthly_charges', 'support_calls', 'churn']].corr()  # compute correlations
plt.figure(figsize=(6, 4))  # set figure size
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')  # heatmap with annotations
plt.title('Correlation Matrix')  # add title
plt.show()  # render heatmap

## Implement Logistic Regression (scikit-learn)
We build a preprocessing + model pipeline, fit, and evaluate with multiple metrics.
(See glossary: Logistic Regression, Threshold, ROC-AUC, Precision/Recall.)

In [ ]:
# Split features and target
X = df.drop(columns=['churn'])  # all independent variables
y = df['churn']  # binary target (0/1)

# Identify column types for preprocessing
num_cols = ['age', 'tenure_months', 'monthly_charges', 'support_calls']  # numeric features
cat_cols = ['contract_type']  # categorical features

# Build transformers
numeric_pipe = Pipeline([  # numeric pipeline
    ('scaler', StandardScaler())  # scale numeric features (helps optimization)
])
categorical_pipe = Pipeline([  # categorical pipeline
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # one-hot encode categories
])

# ColumnTransformer applies each pipeline to respective columns
preprocess = ColumnTransformer([  # assemble preprocessing steps
    ('num', numeric_pipe, num_cols),  # numeric to scaler
    ('cat', categorical_pipe, cat_cols)  # categorical to OHE
])

# Build full pipeline with Logistic Regression classifier
clf = Pipeline([  # model pipeline
    ('prep', preprocess),  # preprocessing step
    ('lr', LogisticRegression(max_iter=1000, class_weight='balanced'))  # use class_weight to counter imbalance
])

# Create stratified train/test split for fair evaluation
X_train, X_test, y_train, y_test = train_test_split(  # split dataset
    X, y, test_size=0.2, stratify=y, random_state=42  # keep class ratio consistent across splits
)

# Fit the model
clf.fit(X_train, y_train)  # train pipeline on training data

# Predict probabilities and classes on test data
y_proba = clf.predict_proba(X_test)[:, 1]  # probability of class 1 (churn)
y_pred = (y_proba >= 0.5).astype(int)  # apply default threshold 0.5 (see glossary: Threshold)

# Compute metrics
acc = accuracy_score(y_test, y_pred)  # accuracy
prec = precision_score(y_test, y_pred)  # precision
rec = recall_score(y_test, y_pred)  # recall
f1 = f1_score(y_test, y_pred)  # F1-score
auc = roc_auc_score(y_test, y_proba)  # ROC-AUC using probabilities

# Display metric summary
print(f'Accuracy: {acc:.3f}')  # print accuracy
print(f'Precision: {prec:.3f}')  # print precision
print(f'Recall: {rec:.3f}')  # print recall
print(f'F1-score: {f1:.3f}')  # print f1
print(f'ROC-AUC: {auc:.3f}')  # print ROC-AUC

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)  # 2x2 confusion matrix
print('Confusion Matrix:
', cm)  # show counts of TP, TN, FP, FN

# Detailed classification report
print('
Classification Report
')  # header
print(classification_report(y_test, y_pred, digits=3))  # precision/recall/F1 per class

In [ ]:
# Plot ROC and Precision-Recall curves for threshold analysis
fpr, tpr, roc_thresholds = roc_curve(y_test, y_proba)  # ROC curve data
precisions, recalls, pr_thresholds = precision_recall_curve(y_test, y_proba)  # PR curve data

fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # two side-by-side plots
# ROC curve
axes[0].plot(fpr, tpr, label=f'AUC = {auc:.3f}')  # plot ROC line with AUC
axes[0].plot([0, 1], [0, 1], 'k--', label='Chance')  # diagonal baseline
axes[0].set_title('ROC Curve')  # add title
axes[0].set_xlabel('False Positive Rate')  # x-label
axes[0].set_ylabel('True Positive Rate')  # y-label
axes[0].legend()  # show legend
# Precision-Recall curve
axes[1].plot(recalls, precisions)  # PR curve plot
axes[1].set_title('Precision-Recall Curve')  # add title
axes[1].set_xlabel('Recall')  # x-label
axes[1].set_ylabel('Precision')  # y-label
plt.tight_layout()  # adjust layout
plt.show()  # render the plots

## Manual Sigmoid + Log Loss on a Mini-batch
We demonstrate the core math: linear score z, sigmoid p, and binary cross-entropy (log loss).
(See glossary: Sigmoid Function, Log Loss.)

In [ ]:
# Select a small subset for manual demonstration
mini = df.sample(5, random_state=1)  # pick 5 random rows for a mini example

# Build a simple numeric-only linear score using a few standardized features
mx = mini[['age', 'tenure_months', 'monthly_charges', 'support_calls']].to_numpy()  # extract numeric matrix
mx_std = (mx - mx.mean(axis=0)) / mx.std(axis=0)  # standardize features to mean 0, std 1
w = np.array([0.2, -0.3, 0.15, 0.25])  # arbitrary weights for illustration (not trained)
b = -0.1  # arbitrary intercept (bias term)
z_manual = mx_std @ w + b  # linear score z = Xw + b
p_manual = 1 / (1 + np.exp(-z_manual))  # sigmoid to convert z to probability p

# Compute binary cross-entropy log loss for the mini-batch
y_true = mini['churn'].to_numpy()  # true labels for the mini-batch
eps = 1e-9  # epsilon to avoid log(0)
log_loss = -np.mean(y_true * np.log(p_manual + eps) + (1 - y_true) * np.log(1 - p_manual + eps))  # average log loss

# Show results
print('z (manual):', np.round(z_manual, 3))  # show linear scores
print('p (manual sigmoid):', np.round(p_manual, 3))  # show probabilities
print('y (true):', y_true)  # show true labels
print(f'Log Loss (manual): {log_loss:.4f}')  # show computed log loss

## Probability Calibration Comparison
We compare calibration of Logistic Regression vs Random Forest on the same test set.
(See glossary: Probability Calibration.)

In [ ]:
# Fit a RandomForestClassifier for comparison
rf = Pipeline([  # build a pipeline mirroring preprocessing
    ('prep', preprocess),  # reuse the same preprocessing steps
    ('rf', RandomForestClassifier(n_estimators=300, random_state=42))  # random forest model
])
rf.fit(X_train, y_train)  # train RF on training data
rf_proba = rf.predict_proba(X_test)[:, 1]  # probability of class 1 from RF

# Compute calibration curves (fraction of positives vs mean predicted value)
lr_frac_pos, lr_mean_pred = calibration_curve(y_test, y_proba, n_bins=10, strategy='uniform')  # LR calibration
rf_frac_pos, rf_mean_pred = calibration_curve(y_test, rf_proba, n_bins=10, strategy='uniform')  # RF calibration

# Plot calibration curves
plt.figure(figsize=(6, 5))  # set figure size
plt.plot([0, 1], [0, 1], 'k--', label='Perfectly Calibrated')  # ideal diagonal
plt.plot(lr_mean_pred, lr_frac_pos, marker='o', label='Logistic Regression')  # LR curve
plt.plot(rf_mean_pred, rf_frac_pos, marker='s', label='Random Forest')  # RF curve
plt.xlabel('Mean Predicted Probability')  # x-label
plt.ylabel('Fraction of Positives')  # y-label
plt.title('Calibration Curves')  # title
plt.legend()  # legend
plt.show()  # render

## Practice Exercises
Complete these tasks; then check your work in the Solutions section.

1) Threshold Tuning (see glossary: Threshold)
- Compute precision, recall, and F1 for thresholds in [0.2, 0.3, ..., 0.8].
- Plot threshold vs each metric.
- Choose an operating point for a retention budget that tolerates some false positives.

2) Feature Impact (see glossary: Odds Ratio)
- Extract coefficients from the trained Logistic Regression.
- Convert coefficients to odds ratios (exp(coef)).
- Interpret two of the strongest effects in plain language.

3) Calibration Check (see glossary: Probability Calibration)
- Compute the Brier score for Logistic Regression and Random Forest.
- Which model is better calibrated on this dataset?

## Solutions (Expand to Reveal)

<details>
<summary><strong>Solution 1: Threshold Tuning</strong></summary>

```python
# Evaluate metrics across thresholds (0.2 to 0.8)
thresholds = np.arange(0.2, 0.81, 0.1)  # define thresholds to test
precisions_list, recalls_list, f1_list = [], [], []  # holders for metrics
for t in thresholds:  # iterate over thresholds
    y_pred_t = (y_proba >= t).astype(int)  # apply threshold t
    precisions_list.append(precision_score(y_test, y_pred_t))  # store precision
    recalls_list.append(recall_score(y_test, y_pred_t))  # store recall
    f1_list.append(f1_score(y_test, y_pred_t))  # store f1
plt.figure(figsize=(7, 4))  # figure size
plt.plot(thresholds, precisions_list, label='Precision')  # precision curve
plt.plot(thresholds, recalls_list, label='Recall')  # recall curve
plt.plot(thresholds, f1_list, label='F1')  # f1 curve
plt.xlabel('Threshold')  # label x-axis
plt.ylabel('Metric')  # label y-axis
plt.title('Metrics vs Threshold')  # title
plt.legend()  # legend
plt.show()  # render
```

</details>

<details>
<summary><strong>Solution 2: Feature Impact (Odds Ratios)</strong></summary>

```python
# Extract LR step from pipeline and compute odds ratios
lr_step = clf.named_steps['lr']  # get logistic regression object
# Build feature names after preprocessing (numeric + one-hot categories)
ohe = clf.named_steps['prep'].named_transformers_['cat'].named_steps['encoder']  # OHE transformer
num_feature_names = num_cols  # numeric names unchanged
cat_feature_names = list(ohe.get_feature_names_out(cat_cols))  # expanded categorical names
feature_names = num_feature_names + cat_feature_names  # full list in order
coefficients = lr_step.coef_.ravel()  # 1D array of coefficients
odds_ratios = np.exp(coefficients)  # convert to odds ratios
impact = pd.DataFrame({'feature': feature_names, 'coef': coefficients, 'odds_ratio': odds_ratios})  # table
impact.sort_values('odds_ratio', ascending=False).head(10)  # view top effects
```

</details>

<details>
<summary><strong>Solution 3: Calibration (Brier Score)</strong></summary>

```python
from sklearn.metrics import brier_score_loss  # import Brier score function
lr_brier = brier_score_loss(y_test, y_proba)  # LR Brier score
rf_brier = brier_score_loss(y_test, rf_proba)  # RF Brier score
print('Brier (LR):', round(lr_brier, 4))  # show LR score
print('Brier (RF):', round(rf_brier, 4))  # show RF score
```

</details>

## Environment and Versions
Record library versions for reproducibility.

In [ ]:
# Show library versions for run-to-run reproducibility
import sklearn  # import scikit-learn to report version
print(f'scikit-learn: {sklearn.__version__}')  # print scikit-learn version
import sys  # Python runtime information
print(f'Python: {sys.version.split()[0]}')  # print Python version number only